# XOR

Can we make `xor` with a net?

In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

`xor(x,y)` is x != y, viz:

In [ ]:
True ^ True, False ^ False, True ^ False, False ^ True

Adopt "CMOS logic levels", where less than 1/2 is False, greater is True. Then

In [ ]:
def flexor(a:float, b:float) -> float:
    return 1.0 if (a < 0.5) ^ (b < 0.5) else 0.0

In [ ]:
flexor(0, 0), flexor(0,1), flexor(1,0), flexor(1,1)

## Support functions
We need some functions implementing nonlinear operations:

In [ ]:
def relu(x):
    #return (lambda v: max(0,v))(x)
    return np.vectorize(lambda v: max(0.0,v))(x)

[(v, relu(v)) for v in np.arange(-2, 2, 0.5)]

In [ ]:
def positive(x):
    return np.vectorize(lambda v: max(0, np.sign(v)))(x)

[(v, positive(v)) for v in np.arange(-2, 2, 0.5)]

In [ ]:
def tanh_gradient(x):
    return 1-np.tanh(x)**2

In [ ]:
def frombits(v) -> int:
    p = 1
    s = 0
    for bit in v:
        s += p * bit
        p <<= 1
    return s

In [ ]:
frombits([1,0,1,1])

## Network implementations

In [ ]:
class Exactor:
    """Calculate exclusive-or using a network"""
    def __init__(self):
        self.m0 = np.array([-0.5, -0.5])
        self.randomize()
        
    def randomize(self):
        "Randomize the matricies"
        self.m1 = np.random.randn(2,2)
        self.m2 = np.random.randn(2)
    
    def make_perfect(self):
        "Set the matricies to a handmade value that gives perfect behavior"
        self.m0 = np.array([-0.5, -0.5])
        self.m1 = np.array([[ 1.0, -1.0],
                            [-1.0,  1.0]])
        self.m2 = np.array([1.0, 1.0])
    
    def ideal(self, a:float, b:float) -> bool:
        "Calculates the ideal return value directly, to provide a reference"
        return 1 if (a > 0.5) ^ (b > 0.5) else 0
        #return np.sign(-a*b)
    
    def netwise(self, a:float, b:float):
        "Calculate a single result using network primitives"
        v = self.net_lin(a, b)
        v = self.p5 = 0 if v < 0.5 else 1
        return v

    def net_ana(self, a:float, b:float):
        "Calculate a single analog result using network primitives"
        v = self.p4 = np.tanh(self.net_lin(a, b))
        return v

    def net_lin(self, a:float, b:float):
        "The network output up to the last linear stage"
        input = np.array([a, b])
        v = self.p0 = self.m0 + input
        v = self.p1 = self.m1 @ input
        v = self.p2 = relu(v)
        v = self.p3 = np.dot(self.m2, v)
        return v

    def __call__(self, a, b):
        "Vectorized calculation of result using network"
        return np.vectorize(self.netwise)(a, b)

    def loss(self):
        "L2 loss function of the network implementation"
        return sum((self.__call__(x,y) - self.ideal(x,y))**2 for x in (-1, 1) for y in (-1, 1))
    
    def aloss(self):
        "L2 loss function of the network analog implementation"
        return sum((self.net_ana(x,y) - self.ideal(x,y))**2 for x in (-1, 1) for y in (-1, 1))
    
    def __repr__(self):
        return F"Exactor m0={self.m0}, m1={self.m1}, m2={self.m2})"
            

### Try out the implementation

In [ ]:
exor = Exactor()
exor
exor.net_lin(1.0,1.0)

In [ ]:
exor.net_ana(1.0,1.0)

In [ ]:
exor.netwise(1,1)

In [ ]:
for x in (0, 1):
    for y in (0, 1):
        print(exor.ideal(x, y), exor(x,y), exor.net_ana(x,y))

In [ ]:
exor.loss(), exor.aloss()

In [ ]:
exor.make_perfect()
print(exor)

In [ ]:
for x in (0, 1):
    for y in (0, 1):
        print(exor.ideal(x, y), exor(x,y), exor.p0, exor.p1, exor.p2, exor.p3, exor.p5)

In [ ]:
exor.loss(), exor.aloss()

In [ ]:
exor.randomize()
exor.loss(), exor.aloss()

### Can we find working matricies by trying random matricies?

In [ ]:
best_loss = 1e300
best_aloss = 1e300
best_repr = ""
for n in range(10000):
    exor.randomize()
    #if n == 6789:
    #    exor.make_perfect()
    loss = exor.loss()
    if best_loss > loss:
        best_loss = loss
        best_repr = repr(exor)
    aloss = exor.aloss()
    if best_aloss > aloss:
        best_aloss = aloss
        best_arepr = repr(exor)
    if loss == 0:
        print(F"Success after {n+1} tries: {exor}")
        break
if exor.loss() > 0:
    print(F"Failure, none of {n+1} random tries worked")
print(f"best net loss:{best_loss}, Best net:{best_repr}")
print(f"best net aloss:{best_aloss}, Best anet:{best_arepr}")

### A less delicate network implementation

FIXME: update to [0,1] logic and analog and so forth

In [ ]:
class Flexor:
    """A more robust XOR
    Returns the sign of (|x-y| - |x+y|) where x, y are inputs"""
    def __init__(self):
        self.randomize()
        
    def randomize(self):
        "Randomize the matricies"
        self.m1 = np.random.randn(4,2)
        self.m2 = np.random.randn(4)
    
    def make_perfect(self):
        "Set the matricies to a handmade value that gives perfect behavior"
        self.m1 = np.array([[ 1.0, -1.0],
                            [-1.0,  1.0],
                            [ 1.0,  1.0],
                            [-1.0, -1.0]])
        self.m2 = np.array([1.0, 1.0, -1.0, -1.0])
    
    def ideal(self, a:float, b:float) -> bool:
        "Calculates the ideal return value directly, to provide a reference"
        return np.sign(-a*b)
    
    def netwise(self, a:float, b:float) -> bool:
        "Calculate a single result using network primitives"
        input = np.array([[a],
                          [b]])
        v = self.p1 = self.m1 @ input
        v = self.p2 = relu(v)
        v = self.p3 = np.dot(self.m2, v)
        v = self.p4 = relu(v)
        v = self.p5 = (-1,1)[int(np.sign(v)[0])]
        return v

    def __call__(self, a, b):
        "Vectorized calculation of result using network"
        return np.vectorize(self.netwise)(a, b)

    def loss(self):
        "L2 loss function of the network implementation"
        return sum((self.__call__(x,y) - self.ideal(x,y))**2 for x in (-1, 1) for y in (-1, 1))
    
    def goodness(self):
        "analog goodness function"
        rv = 0.0
        for x in (-1, 1):
            for y in (-1, 1):
                _ = self.netwise(x, y)
                rv += self.p4 * self.ideal(x,y)
        return rv[0]

    def __repr__(self):
        return F"Flexor(m1={self.m1}, m2={self.m2})"

In [ ]:
flor = Flexor()
flor.m1, flor.m2

In [ ]:
for x in (-1, 1):
    for y in (-1, 1):
        print(flor.ideal(x, y), '\tres:', flor(x,y))

In [ ]:
flor.loss(), flor.goodness()

In [ ]:
flor.make_perfect()

In [ ]:
for x in (-1, 1):
    for y in (-1, 1):
        print(flor.ideal(x, y), flor(x,y))

In [ ]:
flor.loss(), flor.goodness()

In [ ]:
flor.randomize()
flor.loss(), flor.goodness()

### Can we find working matricies by trying random matricies?

In [ ]:
for n in range(10000):
    flor.randomize()
    if flor.loss() == 0:
        print(F"Success after {n+1} tries:\n{flor}")
        break
if flor.loss() > 0:
    print(F"Failure, none of {n+1} random tries worked")

In [ ]:
flor.loss(), flor.goodness()